In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import datetime
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
dataset = pd.read_csv("dataset_2.csv")

In [3]:
dataset.head()

,Unnamed: 0,body_temp,heart_rate,pressure,gyroX,gyroY:,gyroZ,mgr,fall_accident,cardiac_arrest,walkingAsymmetry,Respiratory Rate
0,0,37.7,43,158743.085,0.00,0.00,0.00,7905850400,0,0,0,20
1,1,35.5,108,208412.088,1.49,5.68,0.77,5345941270,0,0,0,19
2,2,36.7,119,149244.192,5.21,2.86,2.35,5117941238,0,0,0,25
3,3,35.4,104,132396.222,3.85,6.21,5.82,7469344503,0,0,0,20
4,4,36.3,72,175302.606,0.00,0.00,0.00,8139834006,1,0,0,18


In [9]:
X = dataset.iloc[:, :8]
# Only detecting cardiac arrest
y = dataset.iloc[:, 9:10]

In [10]:
X.head()

,Unnamed: 0,body_temp,heart_rate,pressure,gyroX,gyroY:,gyroZ,mgr
0,0,37.7,43,158743.085,0.00,0.00,0.00,7905850400
1,1,35.5,108,208412.088,1.49,5.68,0.77,5345941270
2,2,36.7,119,149244.192,5.21,2.86,2.35,5117941238
3,3,35.4,104,132396.222,3.85,6.21,5.82,7469344503
4,4,36.3,72,175302.606,0.00,0.00,0.00,8139834006


In [11]:
y.head()

,cardiac_arrest
0,0
1,0
2,0
3,0
4,0


In [12]:
# Splitting the dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [13]:
# Performing feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [14]:
# Creating the training and testing matrix to run the xgboost model
D_train = xgb.DMatrix(X_train, label = y_train)
D_test = xgb.DMatrix(X_test, label = y_test)

In [15]:
# Params based on grid search
param = {
    'eta': 0.3, 
    'max_depth': 8,  
    'objective': 'multi:softprob',  
    'num_class': 3,
    'gamma' : 0.4,
    'colsample_bytree': 0.7,
    'min_child_weight': 1
    } 

steps = 20  # The number of training iterations

In [16]:
model = xgb.train(param, D_train, steps)

In [18]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
predictions = model.predict(D_test, ntree_limit=model.best_ntree_limit)

prediction_matrix = np.asarray([np.argmax(line) for line in predictions])

print("Precision = {}".format(precision_score(y_test, prediction_matrix, average='macro')))
print("Recall = {}".format(recall_score(y_test, prediction_matrix, average='macro')))
print("Accuracy = {}".format(accuracy_score(y_test, prediction_matrix)))

Precision = 0.7837072611282345
Recall = 0.6659121395036888
Accuracy = 0.9945


In [20]:
print(predictions)

[[0.9974778  0.00136857 0.00115359]
 [0.9974909  0.00135555 0.0011536 ]
 [0.99749446 0.0013896  0.00111599]
 ...
 [0.9974778  0.00136857 0.00115359]
 [0.99758744 0.0012965  0.0011161 ]
 [0.9975374  0.0013466  0.00111604]]
